# Latent Consistency Model using torch.compile with OpenVINO backend
This notebook provides instructions how to run Latent Consistency Model (LCM) using torch.compile with openVINO backend

<img src="https://github.com/openvinotoolkit/openvino_notebooks/assets/105707993/a668529a-e1bd-46c6-9be4-1e6ca705c939"/>


LCMs are the next generation of generative models after Latent Diffusion Models (LDMs). They are proposed to overcome the slow iterative sampling process of Latent Diffusion Models (LDMs), enabling fast inference with minimal steps (from 2 to 4) on any pre-trained LDMs (e.g. Stable Diffusion). To read more about LCM please refer to https://latent-consistency-models.github.io/


### Installation Instructions

This is a self-contained example that relies solely on its own code.

This notebooks is based on https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/latent-consistency-models-image-generation/latent-consistency-models-optimum-demo.ipynb


### Prerequisites
[back to top ⬆️](#Table-of-contents:)

Install required packages

In [ ]:
import sys
!{sys.executable} -m pip install -q "openvino>=2023.3.0"
!{sys.executable} -m pip install -q "accelerate" "diffusers" "ipywidgets" "torch>=2.1.1" "transformers>=4.33.0" --extra-index-url https://download.pytorch.org/whl/cpu

In [2]:
import warnings

warnings.filterwarnings("ignore")

### Using full precision model in CPU with `LatentConsistencyModelPipeline`
[back to top ⬆️](#Table-of-contents:)

Standard pipeline for the Latent Consistency Model(LCM) from Diffusers library is used here. For more information please refer to  https://huggingface.co/docs/diffusers/en/api/pipelines/latent_consistency_models


In [ ]:
from diffusers import LatentConsistencyModelPipeline
import torch
import gc

pipeline = LatentConsistencyModelPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7")
pipeline.unet = torch.compile(pipeline.unet, backend="openvino",options = {"device" :"CPU","model_caching" : True, "cache_dir": "./model_cache"})
pipeline.vae.decode = torch.compile(pipeline.vae.decode, backend="openvino",options = {"device" :"CPU","model_caching" : True, "cache_dir": "./model_cache"})


In [ ]:
prompt = "A cute squirrel in the forest, portrait, 8k"

image = pipeline(prompt=prompt, num_inference_steps=8, guidance_scale=8.0, height=512, width=512).images[0]
image.save("image_standard_pipeline.png")
image

In [ ]:
del pipeline
gc.collect();